In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [ ]:
# ========= CONFIGURAÇÃO DOS PESOS =========
PESOS = {
    # Validacao (float de 1 a 5)
    # Quantidade de pontos por nota:
    # Nota 1: -2 pontos
    # Nota 2: -1 pontos
    # Nota 3: 0 pontos
    # Nota 4: 1 pontos
    # Nota 5: 2 pontos
    'validacao_projeto'
    
    # Satisfacao (integer de 1 a 5)
    # Quantidade de pontos por nota:
    # Nota 1: -0.5 pontos
    # Nota 2: -0.3 pontos
    # Nota 3: 0 pontos
    # Nota 4: 0.3 pontos
    # Nota 5: 0.5 ponto
    'satisfacao_portfolio'
        
        
    # Carga horária: penaliza exponencialmente se a carga for maior que 20 horas/semana
    #
    'carga_horaria': 0.1,             # fator de penalização por cada hora
    # Satisfação com a carga
    'satisfacao_carga': 1.0,
    # Como se sente em relação à carga: mapeamento para um multiplicador
    'sentimento_carga': {
        'ESTOU SATISFEITO': 1.2,
        'SUBALOCADO': 2,
        'SUPERALOCADO': 0.5
    },
    # Saúde mental na PJ: se abaixo de 5, penaliza exponencialmente; se acima, ganha leve bônus
    'saude_mental': {
        'penalidade': 0.2,           # usado em exp(score - 5)
        'bonus': 0.1                 # multiplicador extra para valores acima de 5
    },
    # Número de aprendizagens: cada uma diminui 10% (mínimo 0.8)
    'aprendizagens': 0.1,
    # Assessoria/Liderança: se estiver ativo (valor "INT", "SIM", etc.), penaliza a pontuação
    'assessoria': 0.9,
    # Equipe de PS: se for LIDER, penaliza um pouco
    'equipe_PS': {
        'LIDER': 0.9,
        'ANALISTA': 1.0
    },
    # Semanas até o fim do ciclo atual: se for menor que 2, concede um bônus (ex: 1.1)
    'semanas_ciclo_bonus': 1.1,
    # Penalização para projetos internos: se houver projeto interno em andamento, reduz a pontuação
    'projeto_interno': 0.9
}

In [ ]:
# ========= FUNÇÕES AUXILIARES DE PONTUACAO =========
def disponibilidade_factor(weeks_left):
    """
    Define um fator de disponibilidade com base nas semanas restantes:
      - Se >2 semanas: fator 0.8
      - Entre 1 e 2 semanas: interpolação linear entre 0.8 e 1.0
      - Menor que 1 semana: fator exponencial para aumentar (ex: 1.0 + (1 - exp(-(1 - weeks_left))))
    """
    if pd.isnull(weeks_left):
        return 1.0
    if weeks_left > 2:
        return 0.8
    elif weeks_left >= 1:
        return 0.8 + (2 - weeks_left) * 0.2
    else:
        return 1.0 + (1 - np.exp(-(1 - weeks_left)))
    
def saude_mental_factor(saude):
    """
    Aplica penalização se saúde mental estiver abaixo de 5 e bônus se estiver acima.
    """
    if saude < 5:
        return np.exp((saude - 5) * PESOS['saude_mental']['penalidade'])
    else:
        return 1.0 + PESOS['saude_mental']['bonus'] * (saude - 5)

In [ ]:
# ========= FUNÇÕES AUXILIARES =========
def compute_weeks_left(end_date):
    """Calcula o número de semanas restantes a partir de uma data final."""
    if pd.isnull(end_date):
        return np.nan
    try:
        if not isinstance(end_date, pd.Timestamp):
            end_date = pd.to_datetime(end_date)
        today = pd.Timestamp.today()
        delta = (end_date - today).days / 7.0
        return max(delta, 0)
    except Exception:
        return np.nan

def internal_availability_factor(row):
    """
    Se o membro estiver em Projeto Interno 1 com data futura, calcula um fator de penalização
    baseado nas semanas restantes.
    """
    fim_interno = row.get('Fim do Projeto Interno 1')
    if pd.notnull(fim_interno):
        weeks_internal = compute_weeks_left(fim_interno)
        if weeks_internal > 0:
            return np.exp(-0.2 * weeks_internal)
    return 1.0

In [ ]:
# ========= MEMBROS NÃO ALOCÁVEIS =========
# Defina listas de cargos que não podem ser alocados.
CARGO_NUCLEO_NAO = [
    "Coordenador Comercial", 
    "Coordenador de Inovação Comercial",
    "Coordenador de Inovação de Projetos",
    "Liderança de Outbound",
    "[WI] Lider Comercial"
]
CARGO_MKT_NAO = ["GERENTE"]
CARGO_WI_NAO = ["GERENTE"]

In [ ]:
# ========= FUNÇÃO DE CÁLCULO DO SCORE (MVP) =========
def calcular_score_mvp(row):
    score = 0
    
    # 1. Disponibilidade baseada na data de término do Projeto 1
    # Usa "Fim previsto do Projeto 1 (sem atraso)" se existir, senão "Fim estimado do Projeto 1 (com atraso)"
    end_date = row['Fim previsto do Projeto 1 (sem atraso)'] \
               if pd.notnull(row['Fim previsto do Projeto 1 (sem atraso)']) \
               else row['Fim estimado do Projeto 1 (com atraso)']
    weeks_left = compute_weeks_left(end_date)
    avail_factor = disponibilidade_factor(weeks_left)
    
    # 2. Base score do Projeto 1: combina Satisfação e Validação
    satisfacao = row.get('Satisfação com o Projeto 1', 0)
    validacao = row.get('Validação média do Projeto 1', 0)
    base_score = validacao * PESOS['validacao_projeto']
    score = base_score * avail_factor

    # 3. Bônus da satisfação do portfólio correspondente (usando Portfólio do Projeto 1)
    portfolio = row.get('Portfólio do Projeto 1', None)
    if portfolio:
        peso_port = PESOS['satisfacao_portfolio'].get(portfolio.upper(), 0)
        # A coluna de satisfação com o portfólio pode variar conforme o portfólio
        col_sat_port = f'Satisfação com o Portfólio: {portfolio.upper()}'
        score += row.get(col_sat_port, 0) * peso_port

    # 4. Penalização pela carga horária
    carga = row.get('Carga horária média no último mês hora/semana', 40)
    if carga > 20:
        # Penaliza exponencialmente a partir de 20 horas
        carga_factor = np.exp(-(carga - 20) * PESOS['carga_horaria'])
    else:
        carga_factor = 1.0
    score *= carga_factor

    # 5. Ajuste por Satisfação com a carga (0 a 10)
    sat_carga = row.get('Satisfação com essa carga', 5)
    score *= (sat_carga / 10)

    # 6. Ajuste pelo sentimento em relação à carga
    sentimento = str(row.get('Como se sente em relação à carga', 'Estou satisfeito')).upper()
    score *= PESOS['sentimento_carga'].get(sentimento, 1.0)

    # 7. Ajuste pela Saúde mental na PJ (0 a 10)
    saude = row.get('Saúde mental na PJ', 5)
    score *= saude_mental_factor(saude)

    # 8. Penalização por N° Aprendizagens (cada uma diminui 5% do score, mínimo de 0.8)
    aprendizados = row.get('N° Aprendizagens', 0)
    score *= max(0.8, 1 - PESOS['aprendizagens'] * aprendizados)

    # 9. Penalização por Assessoria/Liderança: se existir indicação (ex: valor "INT")
    assessoria = str(row.get('Assessoria/Liderança', '')).strip().upper()
    if assessoria in ['INT', 'SIM', '1']:
        score *= PESOS['assessoria']

    # 10. Ajuste pela Equipe de PS: se for LIDER, penaliza um pouco
    equipe_ps = str(row.get('Equipe de PS', 'ANALISTA')).strip().upper()
    score *= PESOS['equipe_PS'].get(equipe_ps, 1.0)

    # 11. Bônus se as Semanas até o fim do ciclo atual forem baixas (importante para cargos de 3 meses, etc.)
    semanas_ciclo = row.get('Semanas até o fim do ciclo atual', 4)
    if semanas_ciclo < 2:
        score *= PESOS['semanas_ciclo_bonus']

    # 12. Penalização por Projeto Interno 1 ativo
    score *= internal_availability_factor(row)
    
    return score

In [ ]:
# ========= APLICAÇÃO DO CÁLCULO E ORDENAMENTO =========
df['score'] = df.apply(calcular_score_mvp, axis=1)
df_ordenado = df.sort_values(by='score', ascending=False)

# Exibe as colunas principais para visualização
print("Membros sugeridos (ordenados pela pontuação):")
print(df_ordenado[['Membro', 'score']])